# University of Michigan
## School of Information
**Masters of Applied Data Science**<br>
**Milestone 1**<br>
<br>
Affordable Housing Project
<br><br>
**Submitted by**<br>
>Alan Fehsenfeld<br>Koon Leong Ho<br>George Thio

**Data Visualization**

This Jupyter notebooks produced the visualizations used int he Project Report.

In [7]:
# Import all the libraries used in the visualizations

import pandas as pd
import numpy as np
import geopandas
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os

# Set some key parameters of the libraries used
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

# Set key constants used in the visualization routines
directory=os.getcwd()
properties_data=r'/data/SIADS591_property.pkl'
properties_shp_file = r'/data/municipalities/All Municipalities Geometry and Municipality.shp'
fulton = directory + r'/fulton.html'
preston = directory + r'/preston.html'

#### Geospatial data
These are shapefiles for all the parcels in Kent County.  The data are provided by Kent County by cities and townships.  These were merged into one set of files for the entire county.

The source data geographical information uses "NAD83 State Plane Michigan South (International Feet)" projection.  This will be converted into "latitude and longitude coordinates on the WGS84 reference ellipsoid" projection.

Source: https://www.accesskent.com/GISLibrary/

In [2]:
# Read the shapefiles data to be used for the chloropleth map plots
GRParcelsUnit=geopandas.read_file(directory + properties_shp_file)

# convert into latitude/longitude coordinates
GRParcelsUnit=GRParcelsUnit.to_crs('EPSG:4326')
unit_geodf=GRParcelsUnit.__geo_interface__


In [3]:
# Read in the pickle file containing the attribute of each plot in Kent County
unit_df=pd.read_pickle(directory + properties_data)
unit_df=unit_df.sort_values(by = ['City'], ascending = False)

#### Heatmap
The entire geospatial data has about 240,000 entries.  Two density heapmaps will be plotted:

Plot #1 - This shows the total number of properties of each MSHDA Score in each city.  The darker the color scale, the higher the number of properties with the MSHDA Score.

Plot #2 - This shows the percentage of properties within each city that has that MSHDA Score.  The sum of the percentages within each city will add up to 100%.


In [ ]:
# Preparing the data for the heatmaps
summary_df = unit_df[['City','MSHDA Score', 'Percent']].copy(deep = True)
summary_df = summary_df.groupby(['City', 'MSHDA Score']).agg({'Percent': 'sum'}).reset_index(drop=False)


In [ ]:
heatmap1 = px.density_heatmap(unit_df, x="MSHDA Score", y="City", 
                             title = "Distribution of MSHDA Score within Kent Count",
                             histfunc = "count",
                             color_continuous_scale="Greys",
                             width=900, height=900
                            )
heatmap1.show()

In [ ]:
heatmap2 = px.density_heatmap(unit_df, x="MSHDA Score", y="City", z="Percent",
                             title = "Distribution of MSHDA Score (in Percent) within each City",
                             histfunc = "sum",
                             labels = {"Percent": "%"},
                             facet_row_spacing = 0.3,
                             facet_col_spacing = 0.3,
                             color_continuous_scale="Mint",
                             width=900, height=900
                            )
heatmap2.show()

### Scatter Plot Matrix
The Scatter Plot Matrix (aka SPLOM) shows the relationship between the variables.  Three variables will be explored in this plot:

> a. Size of the property<br>
> b. Estimated cost per Acre (based on SEV)<br>
> c. MSHDA Score<br>


In [ ]:
# Extract the features need for the SPLOM
splom_df = unit_df.loc[:,['City', 'Acres', 'MSHDA Score', 'Per Arce']].copy()

# Set the variables with the limits of the features to exclude the outliers to
# avoid distorting the plot

# sets the limit of the estimated value of the property used by the government to determine the property tax
sev_upper_limit = 5000000

# sets the upper limit to SEV cost per acre
per_acre_upper_limit = 500000

# sets the upper limit of the size of each property
acre_upper_limit = 50

# sets the lower limit of data to be included.   It it is "0", no property will be elimiated from the SPLOM
# on the basis of the MSHDA Score
min_score = 0

# these limits are recognized as the ideal plot size for low cost housing developemnts
ideal_lower_limit = 1
ideal_upper_limit = 10


# Filter the data based on the limits set.

# Remove outliers based on the SEV per Acre limits
per_acre_limit = (splom_df['Per Arce'] < per_acre_upper_limit)
splom_df = splom_df[per_acre_limit]

# Remove outliers based on the size per property
acre_limit = splom_df.Acres < acre_upper_limit
splom_df = splom_df[acre_limit]

# Remove outliers based on the MSHDA Score
MSHDA_limit = splom_df['MSHDA Score'] >= min_score
splom_df = splom_df[MSHDA_limit]

# Categorize the plot size into Small, Ideal, and Large
splom_df['Size']=""
splom_df.loc[splom_df.Acres < ideal_lower_limit, ['Size']]="Small"
splom_df.loc[splom_df.Acres > ideal_upper_limit, ['Size']]="Large"
ideal = (splom_df.Acres >= ideal_lower_limit) & (splom_df.Acres <= ideal_upper_limit)
splom_df.loc[ideal, ['Size']]="Ideal"


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme(style="whitegrid")

plt.figure(figsize=(8000, 8000))

sns.set_theme(style="ticks")
sns.pairplot(splom_df,
             hue_order = ['Small', 'Ideal', 'Large'],
             palette = {'Small': 'gainsboro', 'Ideal': 'r', 'Large': 'powderblue'},
             hue = 'Size',
             diag_kind = 'kde',
             height = 4,
             corner = True
            )
plt.suptitle('Scatter Plot Matrix of plot size, MSHDA Score, and Per Acre cost')
sns.despine()


### Examples
Two properties were selected to demonstrate how the tool can be used.
<br><br>
>The first is a property that has a high MSHDA Score and should be a candidate for Low Cost Housing Development.  It is, in fact, an actual property that was accepted by MSHDA.
<br><br>
>The second property featured has a low MSHDA Score and was not selected for development into Low Cost Housing.

In [ ]:
# print the data for 1450 Fulton St E, Grand Rapids, MI 49503
#
city_of_interest = r'GRAND RAPIDS'
target_apn = '41-14-29-426-036'
rows_buffer = 2

city_df = unit_df.loc[(unit_df.City == city_of_interest), 
                              ['MSHDA Score', 'APN', 'Address', 'City', 
                               'Zip_Code', 'Zip_Code']]. \
                               sort_values(by=['MSHDA Score', 'APN'], ascending = True).set_index('APN')
idx = city_df.index.get_loc(target_apn)
city_df.iloc[idx - rows_buffer: idx + rows_buffer+1]

In [ ]:
# print the data for 1422 Preston Ridge St NW, Grand Rapids mi 49504
#
city_of_interest = r'GRAND RAPIDS'
target_apn = '41-13-14-127-021'
rows_buffer = 2

grand_rapids_df = unit_df.loc[(unit_df.City == city_of_interest), 
                              ['MSHDA Score', 'APN', 'Address', 'City', 
                               'Zip_Code', 'Zip_Code']]. \
                               sort_values(by=['MSHDA Score', 'APN'], ascending = True).set_index('APN')
idx = grand_rapids_df.index.get_loc(target_apn)
grand_rapids_df.iloc[idx - rows_buffer: idx + rows_buffer+1]

### Density Cholopleth Map Plots
This is a plot of all the properties in a specific city.  This is defined in the variable "city_of_interest."  This must match exactly the name of the City in the data file.  Note that the City names are in ALL CAPITAL letters.
<br><br>
The plot may fail if the size of the RAM available in the system used is insufficient.  It does take some time to complete the processing for the plot.

In [5]:
# Define the city of interest and extract the data that is to be plotted.  Only features of relevance to the 
# visualization will be used.  

city_of_interest = r'GRAND RAPIDS'
map_df = unit_df.loc[unit_df.City == city_of_interest, ['ID', 'MSHDA Score', 'APN', 'Address', 'City', 
                                                        'Zip_Code', 'SEV', 'Acres']]


In [8]:
# POSITIVE Exmaple
# Plot map and locality for
# 1450 Fulton St E, Grand Rapids, 49503
# lat/lon of unit extracted from map.google.com

map_center = {'lat': 42.96241, 'lon': -85.63842}
zoom = 15

fig=px.choropleth_mapbox(map_df, geojson=unit_geodf, color='MSHDA Score', locations='ID', 
                         featureidkey='properties.ID', 
                         center= map_center, mapbox_style="carto-positron", 
                         zoom=zoom, hover_data=['MSHDA Score', 'APN', 'Address', 
                                                'City', 'Zip_Code', 'SEV', 'Acres']
                        )

fig.update_layout(margin={'r':0,'t':0,'l':0,'b':0})
fig.write_html(fulton)

In [9]:
# NEGATIVE Exmaple
# Plot map and locality for
# 1422 Preston Ridge NW, Grand Rapids, 49504
# lat/lon of unit extracted from map.google.com

map_center = {'lat': 42.99729, 'lon': -85.70190}
zoom = 15

fig=px.choropleth_mapbox(map_df, geojson=unit_geodf, color='MSHDA Score', locations='ID', 
                         featureidkey='properties.ID', 
                         center= map_center, mapbox_style="carto-positron", 
                         zoom=zoom, hover_data=['MSHDA Score', 'APN', 'Address', 
                                                'City', 'Zip_Code', 'SEV', 'Acres']
                        )

fig.update_layout(margin={'r':0,'t':0,'l':0,'b':0})
fig.write_html(preston)